<a href="https://colab.research.google.com/github/Brandon-Bernal-Alarcon/Notas/blob/main/Machine%20Learning/Evaluacion_de_Modelos/02_Matrices_confusi%C3%B3n_y_m%C3%A9tricas_b%C3%A1sicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Matrices de confusión y métricas básicas de evaluación**

En el notebook anterior se mostró que la precisión (accuracy), aunque ampliamente utilizada, puede ser insuficiente para evaluar el desempeño real de un clasificador, especialmente en problemas con clases desbalanceadas, en este notebook se profundiza en el análisis del rendimiento de modelos de clasificación binaria introduciendo la matriz de confusión y un conjunto de métricas derivadas que permiten distinguir entre distintos tipos de errores.

El objetivo central es comprender cómo y por qué un clasificador se equivoca, y cómo esa información puede utilizarse para seleccionar modelos acordes a los objetivos específicos de una aplicación real.

## **Matriz de confusión**

En un problema de clasificación binaria, cada predicción del modelo cae en una de cuatro categorías posibles:

- Verdadero Negativo (TN): la instancia pertenece a la clase negativa y el clasificador predice negativo.

- Falso Positivo (FP): la instancia pertenece a la clase negativa pero el clasificador predice positivo (error tipo I).

- Falso Negativo (FN): la instancia pertenece a la clase positiva pero el clasificador predice negativo (error tipo II).

- Verdadero Positivo (TP): la instancia pertenece a la clase positiva y el clasificador predice positivo.


Estas cantidades se organizan en la matriz de confusión:

|                     | Pred. Negativa | Pred. Positiva |
|---------------------|---------------:|---------------:|
| **Real Negativa**   | TN             | FP             |
| **Real Positiva**   | FN             | TP             |


La matriz de confusión proporciona una descripción completa del comportamiento del clasificador y constituye la base para la definición de métricas de evaluación más informativas que la precisión



## **Métricas derivadas de la matriz de confusión**

**Precisión (Accuracy)**

$$Acurracy = \frac{TP+TN}{N}$$

Corresponde a la fracción de predicciones correctas y equivale a la suma de la diagonal principal de la matriz de confusión dividida entre el número total de instancias.

**Error de clasificación**

$$Error = \frac{FP + FN}{N} = 1 - Acurracy $$

Mide la fracción total de errores cometidos por el clasificador

**Recall (sensibilidad, tasa de verdaderos positivos)**

$$ Recall = \frac{TP}{TP+TN}$$

Evalúa la capacidad del clasificador para detectar correctamente los casos positivos, es una métrica clave cuando los falsos negativos son especialmente costosos, como en aplicaciones médicas o legales.

**Precisión (Precision)**

$$Precisión = \frac{TP}{TP+FP} $$

Mide qué proporción de las predicciones positivas realizadas por el modelo son correctas. Es fundamental cuando los falsos positivos tienen consecuencias negativas importantes, como en sistemas de recomendación o motores de búsqueda.

**Tasa de falsos positivos y especificidad**


$$ FPR = \frac{FP}{FP + TP} $$

La especificidad se define como:

$$ specificity = 1 - FPR $$

Estas métricas cuantifican el comportamiento del clasificador sobre la clase negativa.

## **Compromiso entre precisión y recall**

una disyuntiva fundamental en Machine LearninG es:

Incrementar la precisión suele reducir el recall.

Incrementar el recall suele reducir la precisión.

La elección del umbral de decisión determina este equilibrio y debe alinearse con el objetivo de la aplicación. No existe un clasificador universalmente óptimo, la métrica adecuada depende del contexto.

## **Puntuación F y F1-score**

Para combinar precisión y recall en una sola métrica se utiliza la puntuación F:

$$F_β = (1+ β^2)*\frac{Precisión*Recall}{(β^2*Presición)+Recall} $$

El caso $𝛽=1$ corresponde a la puntuación F1, que pondera ambas métricas de manera equitativa:

$$F_1 = (2)*\frac{Precisión*Recall}{(Presición)+Recall} $$

Valores de $β<1$ enfatizan la precisión, mientras que valores de $β>1$ enfatizan el recall.

## **Implementación en Python con scikit-learn**

A partir de un clasificador entrenado (por ejemplo, un árbol de decisión), se calculan las métricas de evaluación utilizando sklearn.metrics:

In [7]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

dataset = load_digits()
X, y = dataset.data, dataset.target

for class_name, class_count in zip(dataset.target_names, np.bincount(dataset.target)):
    print(class_name,class_count)


y_binary_imbalanced = y.copy()
y_binary_imbalanced[y_binary_imbalanced != 1] = 0

X_train, X_test, y_train, y_test = train_test_split(X, y_binary_imbalanced, random_state=0)

0 178
1 182
2 177
3 183
4 181
5 182
6 181
7 179
8 174
9 180


In [9]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
tree_predicted = dt.predict(X_test)
confusion = confusion_matrix(y_test, tree_predicted)

print('Decision tree classifier (max_depth = 2)\n', confusion)

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

confusion_matrix(y_test, tree_predicted)
precision_score(y_test, tree_predicted)
recall_score(y_test, tree_predicted)
f1_score(y_test, tree_predicted)

Decision tree classifier (max_depth = 2)
 [[400   7]
 [ 17  26]]


0.6842105263157895

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy: {:.2f}'.format(accuracy_score(y_test, tree_predicted)))
print('Precision: {:.2f}'.format(precision_score(y_test, tree_predicted)))
print('Recall: {:.2f}'.format(recall_score(y_test, tree_predicted)))
print('F1: {:.2f}'.format(f1_score(y_test, tree_predicted)))

Accuracy: 0.95
Precision: 0.79
Recall: 0.60
F1: 0.68


## **Comparación de modelos mediante matrices de confusión**

En este video se comparan explícitamente distintos clasificadores utilizando matrices de confusión, lo que permite analizar no solo cuántos errores comete cada modelo, sino qué tipo de errores.

**Clasificadores ficticios (DummyClassifier)**
Clasificador de la clase mayoritaria

Este clasificador siempre predice la clase más frecuente (clase negativa, 0):

In [17]:
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier


# Negative class (0) is most frequent
dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
y_majority_predicted = dummy_majority.predict(X_test)
confusion = confusion_matrix(y_test, y_majority_predicted)


print('Most frequent class (dummy classifier)\n',
      confusion)

Most frequent class (dummy classifier)
 [[407   0]
 [ 43   0]]


Interpretación:

El clasificador nunca predice la clase positiva.

Obtiene una accuracy aparentemente alta debido al desbalance.

Recall para la clase positiva es 0.

**Clasificador aleatorio proporcional**

Este clasificador predice clases aleatoriamente respetando la proporción de clases del conjunto de entrenamiento:

In [18]:
# produces random predictions w/ same class proportion as training set
dummy_classprop = DummyClassifier(strategy='stratified').fit(X_train, y_train)
y_classprop_predicted = dummy_classprop.predict(X_test)
confusion = confusion_matrix(y_test, y_classprop_predicted)


print('Random class-proportional prediction (dummy classifier)\n', confusion)

Random class-proportional prediction (dummy classifier)
 [[375  32]
 [ 39   4]]


Interpretación:

El modelo detecta algunos positivos, pero con muchos falsos positivos y falsos negativos.

Sirve como línea base mínima para evaluar modelos reales.

**Support Vector Machine (kernel lineal)**

In [19]:
from sklearn.svm import SVC


svm = SVC(kernel='linear', C=1).fit(X_train, y_train)
svm_predicted = svm.predict(X_test)
confusion = confusion_matrix(y_test, svm_predicted)


print('Support vector machine classifier (linear kernel, C=1)\n', confusion)

Support vector machine classifier (linear kernel, C=1)
 [[402   5]
 [  5  38]]


Interpretación:

Excelente balance entre falsos positivos y falsos negativos.

Alta precision y recall para la clase positiva.

Representa una mejora sustancial frente a los clasificadores ficticios.

**Regresión logística**

In [20]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression().fit(X_train, y_train)
lr_predicted = lr.predict(X_test)
confusion = confusion_matrix(y_test, lr_predicted)

print('Logistic regression classifier (default settings)\n', confusion)

Logistic regression classifier (default settings)
 [[401   6]
 [  8  35]]


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Interpretación:

Rendimiento comparable al SVM.

Ligero aumento en falsos negativos.

Advertencia de convergencia indica posible necesidad de escalar datos o aumentar max_iter.

**Árbol de decisión**

In [22]:
from sklearn.tree import DecisionTreeClassifier


dt = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
tree_predicted = dt.predict(X_test)
confusion = confusion_matrix(y_test, tree_predicted)


print('Decision tree classifier (max_depth = 2)\n', confusion)

Decision tree classifier (max_depth = 2)
 [[400   7]
 [ 17  26]]


Interpretación:

Mayor número de falsos negativos.

Menor recall para la clase positiva.

Árbol poco profundo prioriza simplicidad a costa de desempeño.

## **Reporte de clasificación**

Para obtener un resumen completo del desempeño del modelo, se utiliza:

In [12]:
# Combined report with all above metrics
from sklearn.metrics import classification_report

print(classification_report(y_test, tree_predicted, target_names=['not 1', '1']))

              precision    recall  f1-score   support

       not 1       0.96      0.98      0.97       407
           1       0.79      0.60      0.68        43

    accuracy                           0.95       450
   macro avg       0.87      0.79      0.83       450
weighted avg       0.94      0.95      0.94       450



In [21]:
print('Random class-proportional (dummy)\n',
      classification_report(y_test, y_classprop_predicted, target_names=['not 1', '1']))
print('SVM\n',
      classification_report(y_test, svm_predicted, target_names = ['not 1', '1']))
print('Logistic regression\n',
      classification_report(y_test, lr_predicted, target_names = ['not 1', '1']))
print('Decision tree\n',
      classification_report(y_test, tree_predicted, target_names = ['not 1', '1']))

Random class-proportional (dummy)
               precision    recall  f1-score   support

       not 1       0.91      0.92      0.91       407
           1       0.11      0.09      0.10        43

    accuracy                           0.84       450
   macro avg       0.51      0.51      0.51       450
weighted avg       0.83      0.84      0.84       450

SVM
               precision    recall  f1-score   support

       not 1       0.99      0.99      0.99       407
           1       0.88      0.88      0.88        43

    accuracy                           0.98       450
   macro avg       0.94      0.94      0.94       450
weighted avg       0.98      0.98      0.98       450

Logistic regression
               precision    recall  f1-score   support

       not 1       0.98      0.99      0.98       407
           1       0.85      0.81      0.83        43

    accuracy                           0.97       450
   macro avg       0.92      0.90      0.91       450
weighted avg 

El classification report incluye:

- Precision

- Recall

- F1-score

- Soporte (número de instancias por clase)

Este reporte facilita la comparación entre distintos modelos entrenados sobre el mismo conjunto de datos.

## **Conclusiones**

La matriz de confusión es una herramienta esencial para comprender el comportamiento de un clasificador.

Métricas como precision, recall y F1-score proporcionan perspectivas complementarias del desempeño del modelo.

No existe una métrica universalmente óptima; la elección depende del contexto y del costo de los errores.

El análisis basado en la matriz de confusión es fundamental para la selección de modelos en aplicaciones reales.